In [ ]:
import pandas as pd

In [ ]:
# read in participants.tsv taken from /data/origami/nikhil/datasets/sandbox/qpn-new
participants_tsv_source = pd.read_csv("../../participants.tsv", sep="\t")
participants_tsv_source

In [ ]:
# read in demographics_release.csv, created with qpn-release_data-sort.ipynb
demographics_release = pd.read_csv("../../tabular/demographics_release.csv", sep=",")
demographics_release['participant_id'] = 'sub-' + demographics_release['participant_id']
demographics_release

In [ ]:
# filter source participants.tsv by demographics_release.csv participants
participants_tsv_filtered = participants_tsv_source[participants_tsv_source['participant_id'].isin(demographics_release['participant_id'])]
participants_tsv_filtered

In [ ]:
# Find missing participant IDs
missing_ids = demographics_release[~demographics_release['participant_id'].isin(participants_tsv_filtered['participant_id'])]

print("Missing participant IDs:")
print(missing_ids)

In [ ]:
# add subject * because missing in participants_tsv_source. 
# Information taken from /data/origami/nikhil/datasets/sandbox/qpn-new/tabular/assessments/visits.csv and /mri_info/mri_sessions.csv
# NOTE: ID and personal information was removed from this notebook, full information at /host/verges/tank/pd/tmp/qpn-releases/qpn-release_dec-2024/scratch/code

new_row = pd.DataFrame({"participant_id": [""], "age": [], "sex": [""], "group":[""]})
participants_tsv_filtered = pd.concat([participants_tsv_filtered, new_row], ignore_index=True)
participants_tsv_filtered

In [ ]:
# sort new table after demographics_release sorting

participants_tsv_filtered = participants_tsv_filtered.set_index('participant_id').reindex(demographics_release['participant_id']).reset_index()
participants_tsv_filtered 

In [ ]:
# add folder_index column 

participants_tsv_filtered["folder_index"] = demographics_release["folder_index"]
participants_tsv_filtered

In [ ]:
# adjust group values

participants_tsv_filtered['group'] = participants_tsv_filtered['participant_id'].map(demographics_release.set_index('participant_id')['enrollment_group'])
participants_tsv_filtered

In [ ]:
# rename "group" values

participants_tsv_filtered['group'] = participants_tsv_filtered['group'].replace("PD (Parkinson's Disease)/(Maladie de Parkinson)", "PD")
participants_tsv_filtered['group'] = participants_tsv_filtered['group'].replace("AP (Atypical Parkinsonism)/(Parkinsonisme Atypique)", "AP")
participants_tsv_filtered['group'] = participants_tsv_filtered['group'].replace("Healthy control/Contrôle", "control")
# Format folder_index to have leading zeros
participants_tsv_filtered['folder_index'] = participants_tsv_filtered['folder_index'].apply(lambda x: f"{x:03d}")
participants_tsv_filtered

In [ ]:
# export
participants_tsv_filtered.to_csv('participants.tsv', sep='\t', index=False)